In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [2]:
suffix_in = "10"
suffix_out = "10_nn0"
df = pd.read_csv(f'data/preprocessed{suffix_in}.csv')
df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MiscVal,YrSold,MSSubClass^2,MSSubClass LotFrontage,MSSubClass LotArea,MSSubClass OverallQual,MSSubClass OverallCond,MSSubClass YearBuilt,MSSubClass YearRemodAdd,MSSubClass MasVnrArea,MSSubClass BsmtFinSF1,MSSubClass BsmtFinSF2,MSSubClass BsmtUnfSF,MSSubClass TotalBsmtSF,MSSubClass 1stFlrSF,MSSubClass 2ndFlrSF,MSSubClass GrLivArea,MSSubClass BsmtFullBath,MSSubClass BsmtHalfBath,MSSubClass FullBath,MSSubClass HalfBath,MSSubClass BedroomAbvGr,MSSubClass KitchenAbvGr,MSSubClass TotRmsAbvGrd,MSSubClass Fireplaces,MSSubClass GarageYrBlt,MSSubClass GarageCars,MSSubClass GarageArea,MSSubClass WoodDeckSF,MSSubClass OpenPorchSF,MSSubClass EnclosedPorch,MSSubClass ScreenPorch,MSSubClass MiscVal,MSSubClass YrSold,LotFrontage^2,LotFrontage LotArea,LotFrontage OverallQual,LotFrontage OverallCond,LotFrontage YearBuilt,LotFrontage YearRemodAdd,LotFrontage MasVnrArea,LotFrontage BsmtFinSF1,LotFrontage BsmtFinSF2,LotFrontage BsmtUnfSF,LotFrontage TotalBsmtSF,LotFrontage 1stFlrSF,LotFrontage 2ndFlrSF,LotFrontage GrLivArea,LotFrontage BsmtFullBath,LotFrontage BsmtHalfBath,LotFrontage FullBath,LotFrontage HalfBath,LotFrontage BedroomAbvGr,LotFrontage KitchenAbvGr,LotFrontage TotRmsAbvGrd,LotFrontage Fireplaces,LotFrontage GarageYrBlt,LotFrontage GarageCars,LotFrontage GarageArea,LotFrontage WoodDeckSF,LotFrontage OpenPorchSF,LotFrontage EnclosedPorch,LotFrontage ScreenPorch,LotFrontage MiscVal,LotFrontage YrSold,LotArea^2,LotArea OverallQual,LotArea OverallCond,LotArea YearBuilt,LotArea YearRemodAdd,LotArea MasVnrArea,LotArea BsmtFinSF1,LotArea BsmtFinSF2,LotArea BsmtUnfSF,LotArea TotalBsmtSF,LotArea 1stFlrSF,LotArea 2ndFlrSF,LotArea GrLivArea,LotArea BsmtFullBath,LotArea BsmtHalfBath,LotArea FullBath,LotArea HalfBath,LotArea BedroomAbvGr,LotArea KitchenAbvGr,LotArea TotRmsAbvGrd,LotArea Fireplaces,LotArea GarageYrBlt,LotArea GarageCars,LotArea GarageArea,LotArea WoodDeckSF,LotArea OpenPorchSF,LotArea EnclosedPorch,LotArea ScreenPorch,LotArea MiscVal,LotArea YrSold,OverallQual^2,OverallQual OverallCond,OverallQual YearBuilt,OverallQual YearRemodAdd,OverallQual MasVnrArea,OverallQual BsmtFinSF1,OverallQual BsmtFinSF2,OverallQual BsmtUnfSF,OverallQual TotalBsmtSF,OverallQual 1stFlrSF,OverallQual 2ndFlrSF,OverallQual GrLivArea,OverallQual BsmtFullBath,OverallQual BsmtHalfBath,OverallQual FullBath,OverallQual HalfBath,OverallQual BedroomAbvGr,OverallQual KitchenAbvGr,OverallQual TotRmsAbvGrd,OverallQual Fireplaces,OverallQual GarageYrBlt,OverallQual GarageCars,OverallQual GarageArea,OverallQual WoodDeckSF,OverallQual OpenPorchSF,OverallQual EnclosedPorch,OverallQual ScreenPorch,OverallQual MiscVal,OverallQual YrSold,OverallCond^2,OverallCond YearBuilt,OverallCond YearRemodAdd,OverallCond MasVnrArea,OverallCond BsmtFinSF1,OverallCond BsmtFinSF2,OverallCond BsmtUnfSF,OverallCond TotalBsmtSF,OverallCond 1stFlrSF,OverallCond 2ndFlrSF,OverallCond GrLivArea,OverallCond BsmtFullBath,OverallCond BsmtHalfBath,OverallCond FullBath,OverallCond HalfBath,OverallCond BedroomAbvGr,OverallCond KitchenAbvGr,OverallCond TotRmsAbvGrd,OverallCond Fireplaces,OverallCond GarageYrBlt,OverallCond GarageCars,OverallCond GarageArea,OverallCond WoodDeckSF,OverallCond OpenPorchSF,OverallCond EnclosedPorch,OverallCond ScreenPorch,OverallCond MiscVal,OverallCond YrSold,YearBuilt^2,YearBuilt YearRemodAdd,YearBuilt MasVnrArea,YearBuilt BsmtFinSF1,YearBuilt BsmtFinSF2,YearBuilt BsmtUnfSF,YearBuilt TotalBsmtSF,YearBuilt 1stFlrSF,YearBuilt 2ndFlrSF,YearBuilt GrLivArea,YearBuilt BsmtFullBath,YearBuilt BsmtHalfBath,YearBuilt FullBath,YearBuilt HalfBath,YearBuilt BedroomAbvGr,YearBuilt KitchenAbvGr,YearBuilt TotRmsAbvGrd,YearBuilt Fireplaces,YearBuilt Gara

In [3]:
y = df.iloc[:, -1].values
X = df.iloc[:, :-1].values
X.shape, y.shape

((2919, 817), (2919,))

In [4]:
X = MinMaxScaler().fit_transform(X)

In [5]:
train_idx = 1460
X_train, X_test = X[:train_idx], X[train_idx:]
y_train, _ = y[:train_idx], y[train_idx:]

In [6]:
y_train = np.log(y_train)

In [7]:
df_train = pd.DataFrame(np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1))
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,0.064212,0.140098,0.109641,0.413559,0.259231,0.333333,0.0,0.50,0.5,0.375,0.333333,0.461538,0.00,0.346154,0.4,0.368280,0.000000,0.082210,0.000000,0.0,0.0,0.50,0.089636,0.087531,0.014200,0.253165,0.165680,0.242379,0.237008,0.063417,0.125089,0.0,0.027279,0.124512,0.135863,0.187281,0.150258,0.157895,0.000000,0.315789,0.25,0.118421,0.105263,0.167939,0.000000,0.237846,0.157895,0.127620,0.000000,0.054807,0.000000,0.0,0.0,0.235122,0.038799,0.016071,0.131494,0.128445,0.151286,0.151286,0.051134,0.025977,0.0,0.040522,0.029094,0.031195,0.168008,0.055868,0.103834,0.0,0.207668,0.207668,0.155751,0.207668,0.123748,0.000000,0.151286,0.1625,0.080255,0.000000,0.043383,0.000000,0.0,0.0,0.150746,0.001505,0.036059,0.035360,0.034139,0.034139,0.022069,0.016545,0.0,0.0

In [8]:
# rename last column to target
df_train.rename(columns={df_train.columns[-1]: 'target'}, inplace=True)
df_train["target"]

0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: target, Length: 1460, dtype: float64

In [12]:
num_features = X_train.shape[1]
model = Sequential([
    Dense(units=num_features, input_shape=(num_features,), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)),
    Dense(units=512, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)),
    Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)),
    Dense(units=1, activation='linear'),
])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 817)               668306    
                                                                 
 dense_5 (Dense)             (None, 512)               418816    
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,120,019
Trainable params: 1,120,019
Non-trainable params: 0
_________________________________________________________________


# create callback for saving the best model

In [13]:
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self):
        self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs['val_mse']
        if metric_value < self.best:
            self.best = metric_value
            self.best_weights= self.model.get_weights()

class CancelTraining(tf.keras.callbacks.Callback):
    def __init__(self, tol=1e-10):
        self.last_loss = float('inf')
        self.tol = tol

    def on_epoch_end(self, epoch, logs=None):
        loss_delta = abs(logs['val_loss'] - self.last_loss)
        if loss_delta < self.tol:
            print("Model converged. Cancelling training.")
            self.model.stop_training = True
        self.last_loss = logs['val_loss']

In [14]:
save_best_model = SaveBestModel()
history = model.fit(X_train, y_train, epochs=1000, validation_split=0.2, callbacks=[save_best_model, CancelTraining()])
model.set_weights(save_best_model.best_weights)

Epoch 1/1000
37/37 [==============================] - 1s 5ms/step - loss: 277.8111 - mse: 16.1401 - val_loss: 168.7307 - val_mse: 1.1639
Epoch 2/1000
37/37 [==============================] - 0s 4ms/step - loss: 116.1356 - mse: 0.4453 - val_loss: 79.2269 - val_mse: 0.3641
Epoch 3/1000
37/37 [==============================] - 0s 3ms/step - loss: 62.4175 - mse: 0.2834 - val_loss: 48.6183 - val_mse: 0.3157
Epoch 4/1000
37/37 [==============================] - 0s 3ms/step - loss: 40.4402 - mse: 0.2820 - val_loss: 33.1510 - val_mse: 0.3049
Epoch 5/1000
37/37 [==============================] - 0s 3ms/step - loss: 28.3607 - mse: 0.3029 - val_loss: 23.9028 - val_mse: 0.2888
Epoch 6/1000
37/37 [==============================] - 0s 3ms/step - loss: 20.8496 - mse: 0.2982 - val_loss: 17.9450 - val_mse: 0.2818
Epoch 7/1000
37/37 [==============================] - 0s 3ms/step - loss: 15.8887 - mse: 0.2674 - val_loss: 13.9583 - val_mse: 0.3044
Epoch 8/1000
37/37 [==============================] - 0s 3

In [ ]:
pred = model.predict(X_test)

In [ ]:
pd.DataFrame(np.exp(pred), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
                rename(columns={'index': 'id'}).\
                    to_csv(f'data/submission{suffix_out}.csv', index=False)